# Tournament Scheduler Project

## Data Files

In [1]:
# teams.csv
import pandas as pd
from io import StringIO

teams_csv = """TeamID,TeamName
1,Team Alpha
2,Team Beta
3,Team Gamma
4,Team Delta
5,Team Epsilon
6,Team Zeta
7,Team Eta
8,Team Theta
9,Team Iota
10,Team Kappa"""

teams = pd.read_csv(StringIO(teams_csv))
print(teams)

   TeamID      TeamName
0       1    Team Alpha
1       2     Team Beta
2       3    Team Gamma
3       4    Team Delta
4       5  Team Epsilon
5       6     Team Zeta
6       7      Team Eta
7       8    Team Theta
8       9     Team Iota
9      10    Team Kappa


In [3]:
# venues.csv
venues_csv = """VenueID,VenueName,Capacity,AvailableDays
1,Stadium A,5000,"Monday,Wednesday,Friday"
2,Stadium B,3000,"Tuesday,Thursday,Saturday"
3,Stadium C,4000,"Monday,Tuesday,Friday" """

venues = pd.read_csv(StringIO(venues_csv))
print(venues)

   VenueID  VenueName  Capacity              AvailableDays
0        1  Stadium A      5000    Monday,Wednesday,Friday
1        2  Stadium B      3000  Tuesday,Thursday,Saturday
2        3  Stadium C      4000     Monday,Tuesday,Friday 


In [5]:
# constraints.json
import json

constraints_json = {
    "rest_periods": {
        "minimum_hours": 24
    },
    "time_slots": [
        "09:00-11:00",
        "11:30-13:30",
        "14:00-16:00",
        "16:30-18:30",
        "19:00-21:00"
    ],
    "venue_availability": {
        "1": ["Monday", "Wednesday", "Friday"],
        "2": ["Tuesday", "Thursday", "Saturday"],
        "3": ["Monday", "Tuesday", "Friday"]
    },
    "max_consecutive_matches": 2,
    "team_constraints": {
        "avoid_repeats": True,
        "max_matches_per_day": 1
    }
}

print(json.dumps(constraints_json, indent=4))

{
    "rest_periods": {
        "minimum_hours": 24
    },
    "time_slots": [
        "09:00-11:00",
        "11:30-13:30",
        "14:00-16:00",
        "16:30-18:30",
        "19:00-21:00"
    ],
    "venue_availability": {
        "1": [
            "Monday",
            "Wednesday",
            "Friday"
        ],
        "2": [
            "Tuesday",
            "Thursday",
            "Saturday"
        ],
        "3": [
            "Monday",
            "Tuesday",
            "Friday"
        ]
    },
    "max_consecutive_matches": 2,
    "team_constraints": {
        "avoid_repeats": true,
        "max_matches_per_day": 1
    }
}


## Core Files

In [17]:
# population.py
import numpy as np

class Schedule:
    def __init__(self, teams, venues, constraints):
        self.teams = teams
        self.venues = venues
        self.constraints = constraints
        self.schedule = self._initialize_schedule()

    def _initialize_schedule(self):
        num_matches = len(self.teams) * (len(self.teams) - 1) // 2
        return np.random.permutation(num_matches)

    def __repr__(self):
        return f"Schedule({self.schedule})"

In [7]:
# fitness.py
def count_venue_conflicts(schedule, constraints):
    venue_availability = constraints.get("venue_availability", {})
    conflicts = 0
    venue_day_map = {}
    for match in schedule:
        _, _, venue_id, day, timeslot = match
        if (venue_id, day) not in venue_day_map:
            venue_day_map[(venue_id, day)] = []
        venue_day_map[(venue_id, day)].append(timeslot)

    for (venue_id, day), timeslots in venue_day_map.items():
        if day not in venue_availability.get(str(venue_id), []):
            conflicts += len(timeslots)
        else:
            timeslot_counts = {}
            for timeslot in timeslots:
                if timeslot not in timeslot_counts:
                    timeslot_counts[timeslot] = 0
                timeslot_counts[timeslot] += 1
            conflicts += sum(count - 1 for count in timeslot_counts.values() if count > 1)

    return conflicts

In [8]:
# operators.py
def crossover(parent1, parent2):
    point = np.random.randint(1, len(parent1.schedule) - 1)
    child1 = np.concatenate((parent1.schedule[:point], parent2.schedule[point:]))
    child2 = np.concatenate((parent2.schedule[:point], parent1.schedule[point:]))
    return child1, child2

def mutate(schedule, mutation_rate=0.1):
    for i in range(len(schedule)):
        if np.random.rand() < mutation_rate:
            j = np.random.randint(0, len(schedule))
            schedule[i], schedule[j] = schedule[j], schedule[i]
    return schedule

In [15]:
# scheduler.py
from src.ga.population import Schedule
from src.ga.fitness import evaluate_fitness
from src.ga.operators import crossover, mutate

def genetic_algorithm(teams, venues, constraints, generations=100, population_size=50):
    population = [Schedule(teams, venues, constraints) for _ in range(population_size)]
    for generation in range(generations):
        fitness_scores = [evaluate_fitness(ind.schedule, constraints) for ind in population]
        selected = select_parents(population, fitness_scores)
        new_population = []
        for i in range(0, len(selected), 2):
            parent1, parent2 = selected[i], selected[i+1]
            child1, child2 = crossover(parent1, parent2)
            new_population.extend([child1, child2])
        population = [mutate(ind.schedule) for ind in new_population]
        best_fitness = max(fitness_scores)
        print(f"Generation {generation}: Best Fitness = {best_fitness}")
    return population

## Entry Point

In [ ]:
# main.py
from src.ga.scheduler import genetic_algorithm
from src.utils.parser import parse_input_data
from src.utils.visualizer import visualize_schedule
from src.

if __name__ == "__main__":
    teams, venues, constraints = parse_input_data("data/teams.csv", "data/venues.csv", "data/constraints.json")
    final_population = genetic_algorithm(teams, venues, constraints)
    best_schedule = max(final_population, key=lambda s: evaluate_fitness(s.schedule, constraints))
    visualize_schedule(best_schedule)

TypeError: cannot unpack non-iterable numpy.int64 object